In [1]:
#This notebook will be used for the data science capstone project.
import pandas as pd
import numpy as np
!pip -q install folium
import folium # plotting library
print("Hello Capstone Project Course!")


Hello Capstone Project Course!


We will grab the table with beautiful soup.

In [2]:
#We will grab the table with beautiful soup.
from bs4 import BeautifulSoup
import urllib.request #This is so we can pull the url into an HTML
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #This is the url of the table we want to pull
soup = BeautifulSoup(urllib.request.urlopen(url), 'html.parser') #Use Beautiful soup to grab the table
table  = soup.find('table',class_='wikitable sortable') #Store the wikitable as a table

Clean the table into a dataframe that is formatted

In [3]:
#
A=[]
B=[]
C=[]
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) > 0:
     A.append(cells[0].find(text=True))
     B.append(cells[1].find(text=True))
     C.append(cells[2].find(text=True))
df=pd.DataFrame(A,columns=['Postal Code']) #Rename the first collumn    
df['Borough']=B #Rename the second collumn
df['Neighborhood']=C #Rename the third collumn
df_with_deletes=df[~df.Borough.str.contains("Not assigned")] #Get a new dataframe with the not assigned data removed
#Group the postal codes and join the boroughs and neighborhood
df_concat = df_with_deletes.groupby('Postal Code').agg({'Borough': ', '.join,'Neighborhood':', '.join}).reset_index()
#Delete the "\n text"
df_concat["Postal Code"] = df_concat["Postal Code"].str.replace("\n", "")
df_concat["Borough"] = df_concat["Borough"].str.replace("\n", "")
df_concat["Neighborhood"] = df_concat["Neighborhood"].str.replace("\n", "")
df_concat.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
df_concat.shape

(103, 3)

In [25]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')

Merge the two dataframes to get all the info together. Use Postal Code as the identifier.

In [6]:
df_merged = pd.merge(df_concat, df_geo, on = ["Postal Code"], how = 'right')
df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Plot the data to check work.

In [26]:
venues_map = folium.Map(location=[43.8, -79.2], zoom_start=12) 

for i in range(0,len(df_merged)):
    folium.Marker([df_merged.iloc[i]['Latitude'], df_merged.iloc[i]['Longitude']]).add_to(venues_map)

# display map
venues_map